In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
pd.options.mode.chained_assignment = None

In [ ]:
df = pd.read_csv("../input/cleanedhatespeechdataset/cleaned-tweets.csv")

# Make the dataframe rows accessible by loc or iloc with their respective id in the id column 
# df.set_index('id', inplace=True)

In [ ]:
df

In [ ]:
df


In [ ]:
df.label.value_counts()

In [ ]:

from sklearn.model_selection import train_test_split

In [ ]:

X_train, X_val, y_train, y_val = train_test_split(
    df.index.values, 
    df.label.values, 
    test_size = 0.15,
    random_state = 17,
    stratify = df.label.values
)

In [ ]:
df['data_type'] = ['not_set']* df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby([ 'label', 'data_type']).count()

In [ ]:

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].tweet.values,
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type == 'val'].tweet.values,
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False
)
from transformers import BertForSequenceClassification, BertConfig



In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 4 #32
dataloader_train = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size = batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler = SequentialSampler(dataset_val),
    batch_size = 32
)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:

optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, #2e-5 to 5e-5
    eps = 1e-8
)

In [ ]:
epochs = 15
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs
)

In [ ]:
import numpy as np

In [ ]:

from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average="weighted")

In [ ]:
def accuracy_per_class(preds, labels):
    labels_dict_inverse = {v: k for k,v in label_dict.items()}
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print('Class: {}'.format(labels_dict_inverse[label]))
        print('Accuracy: {}\n'.format( len(y_preds[y_preds == label]) / len(y_true)) )

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

In [ ]:

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    print(predictions)
    return loss_val_avg, predictions, true_vals

In [ ]:

for epoch in tqdm(range(1, epochs+1)):
    model.train()
    
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train,
                        desc="Epoch {:1d}".format(epoch),
                        leave=False,
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids'       : batch[0],
            'attention_mask'  : batch[1],
            'labels'          : batch[2]
        }
    
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix( {'training_loss': '{:3f}'.format(loss.item() / len(batch))} )

    #torch.save(model.state_dict(), 'Models/BERT_ft_epoch{}.model'.format(epoch))
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write('Training loss: {}'.format(loss_train_avg))
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    
    tqdm.write('Validation loss: {}'.format(val_loss))
    tqdm.write('f1 score (weighted): {}'.format(val_f1))
    
    
    

In [ ]:
from sklearn.metrics import classification_report
val_true = labels_val.detach().cpu().numpy().reshape((-1,1))
val_preds =  np.argmax(predictions,axis = 1).reshape((-1,1))
print(classification_report(val_true, val_preds))    

In [ ]:
val_preds

In [ ]:
model.save_pretrained('results/tokenizer/')